# Abbaspour - 610398147 - Clustering MNIST
The problem is reduce feature of MNIST dataset images with an autoencoder becomes tuned and trained, and use encoded images (from three set train set, test set and validation set (35% of test set)) to get clustered through four clustering algorithms: K-Means, Mini-Batch K-Means, DBSCAN and HDBSCAN.
Firrst some methods of some libraries are imported for calculations and plottings.

In [ ]:
from clusteval import clusteval
from hdbscan import HDBSCAN 
from hnet import enrichment
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from matplotlib.pyplot import bar, colorbar, figure, gray, imshow, ion, legend, plot, savefig, scatter, show, style, subplot, subplots, title, xlabel, xticks, ylabel
from numpy import argsort, array, cumsum, matmul, prod, random, reshape, round, sort, sqrt, unique, vstack, where
from os import environ
from pandas import read_csv
from plotly.graph_objs import Figure, Layout, Scatter3d
from plotly.offline import iplot
from seaborn import histplot
from scipy.linalg import eigh
from sklearn.cluster import DBSCAN, KMeans, MiniBatchKMeans
# from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, adjusted_mutual_info_score, adjusted_rand_score, calinski_harabasz_score, completeness_score, confusion_matrix, ConfusionMatrixDisplay, davies_bouldin_score, homogeneity_score, mutual_info_score, pairwise_distances_argmin, silhouette_score, rand_score, v_measure_score
from sklearn.model_selection import GridSearchCV
# from sklearn.preprocessing import StandardScaler
from time import time
from warnings import filterwarnings
filterwarnings("ignore")
environ["KMP_DUPLICATE_LIB_OK"], environ["TF_CPP_MIN_LOG_LEVEL"], environ["TF_ENABLE_ONEDNN_OPTS"] = "TRUE", "3", "0"

# Loading Dataset & Defining Variables
Train set and test set are read as data frames.
As a preprocessing, values are divided by 255 as they're in range (0, 255) as a normalization.

In [ ]:
# Loading & normalizing the dataset
trainSet, testSet = read_csv("mnist_train.csv"), read_csv("mnist_test.csv")
df = trainSet._append(testSet, ignore_index = True)
X_train, X_test, y_train, y_test, Optimizers, Activations, Architectures, Initializers, learningRates, Epochs, batchSizes, Colors, lossFunction, Accuracy, Loss, validationAccuracy, validationLoss, Title, Alpha, localPopulation, Numbers, numberIterations, numberGenerations, Fitness, Parameters, Tuner = trainSet.drop(["label"], axis = 1).astype("float32") / 255., testSet.drop(["label"], axis = 1).astype("float32") / 255., trainSet["label"].astype("int"), testSet["label"].astype("int"), {Adam: "Adam" , Nadam: "Nadam", SGD: "SGD"}, ["linear", "relu", "tanh"], [(128, 16), (500, 100, 30), (128, 64, 32)], ["glorot_normal", "glorot_uniform", "zero"], [0.001, 0.01, 0.1, 0.2, 0.25, 0.3, 0.5, 0.8], [1, 10, 15, 25, 50], [256, 300, 500, 1000, 5000, 10000, 30000, 60000], ["black", "blue", "cyan", "green", "magenta", "maroon", "purple", "red", "teal", "yellow"], "mean_squared_error", "accuracy", "loss", "val_accuracy", "val_loss", "Model Evaluation", 0.6, 1, 10, 1, 1, [], [], {}
dfX_train, optimizerNames, localBound, number_test_samples, Beta = X_train, list(Optimizers.values()), X_train.shape[1], len(X_test), 1 - Alpha

# Defining the encoder input & decoder input dimentions.
Image, Length, Start = Input(shape = (localBound,)), int(sqrt(localBound)), int(number_test_samples * 0.35)

# 0.35% of test samples are chosen as validation samples
validationSet = testSet.iloc[Start : number_test_samples - Start]
y_val, X_val, arrX_train, arrX_test, arrtestSet = validationSet["label"], validationSet.drop(["label"], axis = 1), X_train.to_numpy().reshape(-1, 28, 28), X_test.to_numpy().reshape(-1, 28, 28), testSet.to_numpy()
print(f"There're {len(X_train)} samples of {Length} X {Length} images as train samples and {number_test_samples} samples as test samples.")

# Tuning Autoencoder
Target is to minimizer Mean Square Error value as error function.
Autoencoder is fine tuned by searching step by step between three optimizers 'Adam', 'Nadam' & 'SGD', three architectures (128, 16), (500, 100, 30) and (128, 64, 32), weight initializers methods 'Glorot Normal', 'Glorot Uniform' and 'Zero' and activation functions 'Linear', 'Relu' and 'Tanh'.
When best optimizer is found by searching, model would be search for good learning rate between candidate learning rates.
In tuning, batch size equals to 60000 which is the number of train samples and search for each tuple of hyperparameters comes done in 1 epoch due to time limitation and also there isn't sensibe differences between searching in different epochs but getting closer and closer to overfitting event.

In [ ]:
# Fine tuning hyperparameters
for Optimizer in optimizerNames:
    for Architecture in Architectures:
        decoderInput, number_hidden_layers = Input(shape = (Architecture[-1],)), len(Architecture) - 1
        for Initializer in Initializers:
            for Activation in Activations:
                # Define & build the encoder & decoder
                Encoded, Decoded = Dense(Architecture[0], activation = Activation, kernel_initializer = Initializer)(Image), Dense(Architecture[-2], activation = Activation, kernel_initializer = Initializer)(decoderInput)
                for hiddenLayer in range(1, number_hidden_layers):
                    Encoded, Decoded = Dense(Architecture[hiddenLayer], activation = Activation, kernel_initializer = Initializer)(Encoded), Dense(Architecture[-(hiddenLayer + 2)], activation = Activation, kernel_initializer = Initializer)(Decoded)

                Encoder, Decoder = Model(Image, Dense(Architecture[-1], activation = Activation)(Dense(Architecture[-2], activation = Activation)(Encoded)), name = "Encoder"), Model(decoderInput, Dense(localBound, activation = "sigmoid", kernel_initializer = Initializer)(Decoded), name = "Decoder")
                
                # Build main autoencoder
                Autoencoder = Model(Image, Decoder(Encoder(Image)))

                # Compile the autoencoder
                Autoencoder.compile(loss = lossFunction, metrics = ["accuracy"], optimizer = Optimizer)

                # Fit the data on the Autoencoder
                Autoencoder = Autoencoder.fit(X_train, X_train, epochs = 1, batch_size = 60000, shuffle = True, validation_data = (X_val, X_val), verbose = 0)
                Results = Autoencoder.history
                Tuner[(Optimizer, Architecture, Initializer, Activation)] = Alpha * (Results[validationAccuracy][0] - Results[validationLoss][0]) + Beta * (Results[Accuracy][0] - Results[Loss][0])

Optimizer, Architecture, Initializer, Activation = max(Tuner, key = Tuner.get)
optimizerName = Optimizer
Optimizer, decoderInput, number_hidden_layers, Tuner = list(Optimizers.keys())[optimizerNames.index(optimizerName)], Input(shape = (Architecture[-1],)), len(Architecture) - 1, {}

# Define & build well-tuned encoder & decoder
Encoded, Decoded = Dense(Architecture[0], activation = Activation, kernel_initializer = Initializer)(Image), Dense(Architecture[-2], activation = Activation, kernel_initializer = Initializer)(decoderInput)
for hiddenLayer in range(1, number_hidden_layers):
    Encoded, Decoded = Dense(Architecture[hiddenLayer], activation = Activation, kernel_initializer = Initializer)(Encoded), Dense(Architecture[-(hiddenLayer + 2)], activation = Activation, kernel_initializer = Initializer)(Decoded)

Encoder, Decoder = Model(Image, Dense(Architecture[-1], activation = Activation)(Dense(Architecture[-2], activation = Activation)(Encoded)), name = "Encoder"), Model(decoderInput, Dense(localBound, activation = "sigmoid", kernel_initializer = Initializer)(Decoded), name = "Decoder")
# Search for good-enough learning rate
for learningRate in learningRates:
    # Build main autoencoder
    Autoencoder = Model(Image, Decoder(Encoder(Image)))

    # Compile the autoencoder
    Autoencoder.compile(loss = lossFunction, metrics = ["accuracy"], optimizer = Optimizer(learning_rate = learningRate))

    # Fit the data on the Autoencoder
    Autoencoder = Autoencoder.fit(X_train, X_train, epochs = 1, batch_size = 60000, shuffle = True, validation_data = (X_val, X_val), verbose = 0)
    Results = Autoencoder.history
    Tuner[learningRate] = Alpha * (Results[validationAccuracy][0] - Results[validationLoss][0]) + Beta * (Results[Accuracy][0] - Results[Loss][0])

learningRate = max(Tuner, key = Tuner.get)
print(f"\nOptimal optimizer, architecture, primary weights initializer method, activation functions for hidden layers and learning rate are {optimizerName}, {Architecture}, {Initializer}, {Activation} and {learningRate}, respectively.\n\nTrainig optimal autoencoder")

# Training Autoencoder
Autoencoder is now constructed with chosen values of hyper parameters.

In [ ]:
# Build main autoencoder
Autoencoder = Model(Image, Decoder(Encoder(Image)))

# Compile the autoencoder
Autoencoder.compile(loss = lossFunction, metrics = ["accuracy"], optimizer = Optimizer(learning_rate = learningRate))
print(Autoencoder.summary())

# Saving model
Autoencoder.save("Autoencoder.keras")

# Fit the data on the Autoencoder
# Test set is now the validation set of model!
autoencoderHistory = Autoencoder.fit(X_train, X_train, epochs = 10, batch_size = 256, shuffle = True, validation_data = (X_test, X_test), verbose = 0)


# Autoencoder Evaluation Plotting
Final loss value and accuracy of model's perfomance on train set and test set is plotted.

In [ ]:
# Plot the training history
figure(figsize = (13, 5))
ion()
plot(autoencoderHistory.history[Accuracy])
plot(autoencoderHistory.history[validationAccuracy])
plot(autoencoderHistory.history[Loss])
plot(autoencoderHistory.history[validationLoss])
title(Title)
ylabel("Loss")
xlabel("Epochs")
legend(["Train Accuracy", "Test Accuracy", "Train Loss", "Test Loss"], loc = "upper right")
savefig(Title, dpi = 1200)
show()

# Datasets Encoding
Datasets get encoded in their corresponding batch sizes (small enough to get better accuracies and also save time!) which are 700, 300 and 256 for train set, test set and validation set, respectively.

In [ ]:
# Encoding images
X_train, X_test, X_val, Title = Encoder.predict(X_train, batch_size = 700), Encoder.predict(X_test, batch_size = 300), Encoder.predict(X_val, batch_size = 256), "Elbow Method"

# Which number of Clusters is optimal?
According to following plot, 10 number of clusters is optimal one.

In [ ]:
# Which number of clusters should be chosen as best one
figure(figsize = (13, 5))
ion()
plot(range(1, 11), [KMeans(algorithm = "elkan", init = "k-means++", max_iter = 1000, n_clusters = numberCluster, n_init = "auto", random_state = 42).fit(X_train).inertia_ for numberCluster in range(1, 11)], marker = "o")
title(Title)
xlabel("Number of Clusters")
ylabel("Inertia")
savefig(Title, dpi = 1200)
show()

# K-Means & Mini-Batch K-Means Clustring Algorithms
Two algorithms from 'K-Means' family are implemented (by means of 'scikit-learn' library) and selected centroids are plotted (which shows that accuracy is very low; as it'll be calculated)
These models once get tuned by 'GridSearchCV' method.
Many measures as clustering algorithm evaluation metrics are calculated and printed.
According to results for once (illustrated in 'Evaluations' folder), did better in many metrics.

In [ ]:
# Define, fit (train) & evaluate four models
KAverages = {"K-Means Clustering Algorithm": (KMeans, GridSearchCV(KMeans(), param_grid = {"algorithm": ["elkan", "lloyd"], "init": ["k-means++", "random"], "max_iter": [100, 200, 500, 1000], "n_clusters": [10], "n_init": ["auto"]}, n_jobs = -1, cv = 5)), "Mini-Batch K-Means Clustering Algorithm": (MiniBatchKMeans, GridSearchCV(MiniBatchKMeans(), param_grid = {"init": ["k-means++", "random"], "max_iter": [100, 200, 500, 1000], "n_clusters": [10], "n_init": ["auto"]}, n_jobs = -1, cv = 5))}
for clustererName in KAverages:
    print(f"\n{clustererName} Modeling Started!")
    # Using Bayes Search approach for tuning model
    Clusterer, Title = KAverages[clustererName], f"{clustererName} Confusion Matrix"
    model = Clusterer[1]
    startTime = time()
    # Obtaining best paramteres by using splitted validation data
    model.fit(X_val)
    Duration = time() - startTime # Calculating how long fitting validation data takes
    print(f"\nModel's best score is:\n{model.best_score_}\n\nModel's best parameters are:\n{model.best_params_}")
    model = model.best_estimator_ # = Model(Optimal Parameters)
    print(f"\n{model}")
    # Evaluation Time!
    model.fit(X_train)
    Duration, Labels, Centroids, realLabels = time() - Start, model.labels_, model.cluster_centers_, len(unique(y_train))
    y_pred, nearestInstances, clusterIndexes, clustered_data_points, numberClusters = model.fit_predict(X_test), pairwise_distances_argmin(Centroids, X_train), [[] for realLabel in range(realLabels)], [[] for realLabel in range(realLabels)], len(unique(Labels))
    Trace = [[] for Cluster in range(numberClusters)]
    
    # Declaring number of datapoints in individual clusters
    for Index, Label in enumerate(Labels):
        for Number in range(numberClusters):
            if Label == Number:
                clustered_data_points[Number].append(Index)
    
    for Cluster in range(numberClusters):
        print(f"No. of items in {Cluster}th cluster: {len(clustered_data_points[Cluster])}")
    print(f"\nAll {sum([len(dataPoints) for dataPoints in clustered_data_points])} training data points are clustered.")

    """
    # Plotting clustered training samples in 3D
    for Cluster in range(numberClusters):
        Members = (clusterIndexes[Cluster])
        Trace[Cluster] = Scatter3d(x = arrX_train[Members, 0], y = arrX_train[Members, 1], z = arrX_train[Members, 2], mode = "markers", marker = dict(size = 2, color = Colors[Cluster]), name = f"Cluster {Cluster}", hoverinfo = "text")
        layout = Layout(title = "3D Scatter Plot")
    iplot(Figure(data = [Trace[0], Trace[1], Trace[2], Trace[3], Trace[4], Trace[5], Trace[6], Trace[7], Trace[8], Trace[9]], layout = layout))

    uniqueLabels = unique(y_pred)
    # Plotting the clusters
    figure(figsize = (8, 8)) 
    for i in unique_labels:
        scatter(arrtestSet["label" == i, 0], arrtestSet["label" == i, 1], label = i)
    scatter(Centroids[:, 0], Centroids[:, 1], marker = "x", s = 169, linewidths = 3, color = "k", zorder = 10) 
    legend() 
    show()
    # Use sklearn function to calculate the nearest data instance 
    # for each cluster center. The function returns the indices of 
    # the nearest instances.
    """
    numberClusters = len(unique(y_pred))
    # Plotting selected centroids by implied algorithm
    Fig, Axes = subplots(1, numberClusters)
    for Centroid in range(numberClusters):
        # Plot the images
        Axes[Centroid].imshow(arrX_train[nearestInstances[Centroid]], cmap = "grey")

        # Styling
        Axes[Centroid].set_yticks([])
        Axes[Centroid].set_xticks([])
    title(f"Centroids\nProvided by {clustererName}")
    savefig(f"Centroids by {clustererName}", dpi = 1200)
    show()

    # Evaluation metrics
    print(f"Model is trained in {Duration} UTC\n\nMeasures:\nAccuracy: {accuracy_score(y_test, y_pred)}\nAdjusted Mutual Information (AMI) Score: {round(adjusted_mutual_info_score(y_test, y_pred) * 100, 2)}\nAdjusted Rand Index (ARI) Score: {round(adjusted_rand_score(y_test, y_pred) * 100, 2)}\nCalinski Harabasz Score: {calinski_harabasz_score(X_test, y_pred)}\nCompleteness Score: {completeness_score(y_test, y_pred)}\nDavies Bouldin Score: {davies_bouldin_score(X_test, y_pred)}\nHomogeneity Score: {homogeneity_score(y_test, y_pred)}\nMutual Info Score: {mutual_info_score(y_test, y_pred)}\nRand Score: {rand_score(y_test, y_pred)}\nSSE of random data's cluster results: {model.inertia_}\nSilhouette Score: {silhouette_score(X_test, y_pred, metric = 'euclidean', sample_size = 300)}\nV Measure: {v_measure_score(y_test, y_pred)}")
    ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, y_pred)).plot()
    title(Title)
    savefig(Title, dpi = 1200)
    show()

# DBSCAN Clustering Algorithm
Density-Based Spatial Clustering of Applications with Noise (DBSCAN) clustering algorithm is implemented in two forms.

In [ ]:
# Compute the clustering using DBSCAN
print("\nDensity-Based Spatial Clustering of Applications with Noise (DBSCAN) Clustering Algorithm")
Title = "DBSCAN Clustering Algorithm Confusion Matrix"
startTime = time()
model = DBSCAN(algorithm = "brute", eps = 30, metric = "euclidean", min_samples = 50)
model.fit(X_train)
y_pred = model.fit_predict(X_test)
Duration = time() - startTime
# Evaluation metrics
print(f"Model is trained in {Duration} UTC\n\nMeasures:\nAccuracy: {accuracy_score(y_test, y_pred)}\nAdjusted Mutual Information (AMI) Score: {round(adjusted_mutual_info_score(y_test, y_pred) * 100, 2)}\nAdjusted Rand Index (ARI) Score: {round(adjusted_rand_score(y_test, y_pred) * 100, 2)}\nCompleteness Score: {completeness_score(y_test, y_pred)}\nHomogeneity Score: {homogeneity_score(y_test, y_pred)}\nMutual Info Score: {mutual_info_score(y_test, y_pred)}\nRand Score: {rand_score(y_test, y_pred)}\nV Measure: {v_measure_score(y_test, y_pred)}")
ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, y_pred)).plot()
title(Title)
savefig(Title, dpi = 1200)
show()

# Silhouette cluster evaluation.
model = clusteval(evaluate = "silhouette")
# In case of using dbindex, it is best to clip the maximum number of clusters to avoid finding local minima.
model = clusteval(evaluate = "dbindex", max_clust = 10)
# Derivative method.
model = clusteval(evaluate = "derivative")
# DBscan method.
model = clusteval(cluster = "dbscan", params_dbscan = {"epsres": 100, "norm": True})

Results = model.fit(X_train)
# Clustering labels
print(Results["labx"])

model.plot()
model.plot_silhouette()
model.scatter()
model.dendrogram()

# HDBSCAN Clustering Algorithm
Hierarchical Density-Based Spatial Clustering of Applications with Noise (HDBSCAN) clustering algorithm is implemented in two forms.

In [ ]:
# Compute the clustering using HDBSCAN
print("\nHierarchical Density-Based Spatial Clustering of Applications with Noise (HDBSCAN) Clustering Algorithm")
Title = "HDBSCAN Clustering Algorithm Confusion Matrix"
startTime = time()
model = HDBSCAN(algorithm = "best", alpha = 1.0, approx_min_span_tree = True, cluster_selection_method = "eom", gen_min_span_tree = False, leaf_size = 40, metric = "euclidean", min_cluster_size = 5, min_samples = 50)
model.fit(X_train)
y_pred = model.fit_predict(X_test)
Duration = time() - startTime
# Evaluation metrics
print(f"Model is trained in {Duration} UTC\nMeasures:\nAccuracy: {accuracy_score(y_test, y_pred)}\nAdjusted Mutual Information (AMI) Score: {round(adjusted_mutual_info_score(y_test, y_pred) * 100, 2)}\nAdjusted Rand Index (ARI) Score: {round(adjusted_rand_score(y_test, y_pred) * 100, 2)}\nCalinski Harabasz Score: {calinski_harabasz_score(X_test, y_pred)}\nCompleteness Score: {completeness_score(y_test, y_pred)}\nDavies Bouldin Score: {davies_bouldin_score(X_test, y_pred)}\nHomogeneity Score: {homogeneity_score(y_test, y_pred)}\nMutual Info Score: {mutual_info_score(y_test, y_pred)}\nRand Score: {rand_score(y_test, y_pred)}\nSilhouette Score: {silhouette_score(X_test, y_pred, metric = 'euclidean', sample_size = 300)}\nV Measure: {v_measure_score(y_test, y_pred)}")
ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, y_pred)).plot()
title(Title)
savefig(Title, dpi = 1200)
show()

model = clusteval(cluster = "hdbscan")
# Evaluate
Results = model.fit(X_train)
print(Results)
Labels = Results["labx"]
print(Labels)

# Make plot of the evaluation
model.plot()
# Make scatter plot using the first two coordinates. 
model.scatter(X_train)

# Compute the enrichment of the cluster labels with the dataframe df
print(enrichment(dfX_train, Labels))

# Results
Accuracy of HDBSCAN clustering algorithm on encoded images was the best (approximately equals 0.01). Evaluation metrics got too much small values and near to each other while autoencoder was trained with 'Nadam' optimizer, (500, 100, 30) architecture and with linear activation functions, as a study case.

# Challenges
Main challenge was to fine tune autoencoder automatically (especially for different architectures) sync encoded images for getting clustered with four different clustering algorithms!